In [5]:
import json
import re
import random

data = {}
transfer_stations = []

In [6]:
with open('subway.txt', 'r') as f:
    for line_num, l in enumerate(f):
        stations = re.split(r'\s+-\s+|\n', l.strip('\n'))

        for index, station in enumerate(stations):
            data.setdefault(station, {})
            data[station].setdefault(line_num + 1, [])

            if index > 0:
                data[station][line_num + 1].append((stations[index - 1], str(line_num + 1), random.randint(2, 5)))
            if index < len(stations) - 1:
                data[station][line_num + 1].append((stations[index + 1], str(line_num + 1), random.randint(2, 5)))

In [7]:
for station in data.keys():

    if len(data[station]) == 1:
        continue

    for line in data[station].keys():
        for transfer_line in data[station].keys():

            if line == transfer_line:
                continue

            data[station][line].append((station, str(transfer_line), random.randint(1, 4)))

In [8]:
with open('subway(unequal).json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

### 실제 역간거리 반영

In [1]:
import pandas as pd
distancedf = pd.read_csv("./역간거리.csv",header=None)
transferdf = pd.read_csv("./환승역모음.csv")

In [2]:
transferdf.head()

,호선,환승역명,환승노선,환승소요시간(초)
0,1,시청,2,84
1,1,신길,5,202
2,1,신도림,2,68
3,1,창동,4,70
4,2,성수,2,19


In [3]:
import json
import re
import random

data = {}
transfer_stations = []

In [4]:
def get_traveltime(station1:str,station2:str) -> float:
    df = distancedf[((distancedf[1]==station1) & (distancedf[2]==station2)) | ((distancedf[1]==station2) & (distancedf[2]==station1))]
    return df[3].values[0]/60

def get_transfertime(station:str,line:str,transfer_line:str) -> float:
    if transfer_line[-1] == line[-1]:
        return 2.0
    df = transferdf[transferdf["환승역명"]==station]
    df = df[(df["호선"]==int(line[-1])) & (df["환승노선"]==int(transfer_line[-1])) | (df["환승노선"]==int(line[-1])) & (df["호선"]==int(transfer_line[-1]))]
    return df["환승소요시간(초)"].values[0]/60 + 2.0

In [5]:
with open('real_subway.txt', 'r') as f:
    for _, l in enumerate(f):
        line, l = l.split(",")
        stations = re.split(r'\s+-\s+|\n', l.strip('\n'))

        for index, station in enumerate(stations):
            data.setdefault(station, {})
            data[station].setdefault(line, [])
            try:
                if index > 0:
                    data[station][line].append((stations[index - 1], line, get_traveltime(station,stations[index-1])))
                if index < len(stations) - 1:
                    data[station][line].append((stations[index + 1], line, get_traveltime(station,stations[index+1])))
            except:
                print(index)
                print(station)
                print(stations[index+1])
                raise

In [6]:
for station in data.keys():

    if len(data[station]) == 1:
        continue

    for line in data[station].keys():
        for transfer_line in data[station].keys():

            if line == transfer_line:
                continue
            data[station][line].append((station, str(transfer_line), get_transfertime(station,line,transfer_line)))

In [7]:
get_transfertime("광운대","경인선 1","경부선 1")

2.0

In [7]:
with open('subway(real).json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)